In [1]:
from WickContractions.wick.contract import *
from WickContractions.laph.diagram import *

from utilities import *

from IPython.display import display, Math

pprint = lambda o : display(Math(str(o)))

## Operators

At the moment we are hardcoding in the operators.  We will want to read in a dictionary of spin matrices, as well as isospin flavor combinations.

In [2]:
# TODO get input from operator code for Isospin/momentum combinations.

In [29]:
# load definition of the building blocks of the operators.
from udud_elementals import *
import numpy as np

gammas = [('buchoff',0)]
gammas += np.array([[('A1g',i) for i in range(1)] for row in range(1)]).reshape(1,2).tolist()
gammas += np.array([[('Eg{}'.format(row),i) for i in range(3)] for row in range(2)]).reshape(6,2).tolist()

print(gammas)

gammaName = lambda irrep,i : "\\Gamma^{{ ({},{}) }}".format(irrep, i)

aOps=[]
cOps=[]
for g in gammas:
    aOps.append(Operator([baryonSink(1,['u','u','u','u'],'x_0',gammaName(g[0],g[1]))]))
    cOps.append(Operator([baryonSource(1,['u','u','u','u'], 'x_1',gammaName(g[0],g[1]))]))

[('buchoff', 0), ['A1g', '0'], ['Eg0', '0'], ['Eg0', '1'], ['Eg0', '2'], ['Eg1', '0'], ['Eg1', '1'], ['Eg1', '2']]


## Wick Contractions

Now we run the wick contractions giving us essentially a list of tensor contractions to be performed.

In [30]:
diagrams=[]
for aOp in aOps:
    for cOp in cOps:
        for d in contract(aOp, cOp).diagrams:
            diagrams.append(d)
print("Need {} diagrams".format(len(diagrams)))

Need 1536 diagrams


In [31]:
laphDiagrams = [LDiagram(d) for d in diagrams]
simplify_laphDiagrams(laphDiagrams)
#for d in laphDiagrams:
#    pprint(d)

In [32]:
for b in laphDiagrams[0].Bblocks:
    pprint(b)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [33]:
for t in laphDiagrams[0].Tblocks:
    pprint(t)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [34]:
#allBaryonTensors = find_all_baryon_tensors(laphDiagrams)
# temporary TODO make smarter version of the rest of this notebook
allBaryonTensors={}
idx=0
for g in gammas:
    allBaryonTensors['B(x_1,t_i,\\Gamma^{{ ({},{}) }})'.format(g[0],g[1])]=idx
    allBaryonTensors['B^*(x_0,t_i,t_f,\\Gamma^{{ ({},{}) }})'.format(g[0],g[1])]=idx+1
    idx+=2

allBaryonTensorsBack={v: k for k,v in allBaryonTensors.items()}

In [35]:
allBaryonTensors

{'B(x_1,t_i,\\Gamma^{ (buchoff,0) })': 0,
 'B^*(x_0,t_i,t_f,\\Gamma^{ (buchoff,0) })': 1,
 'B(x_1,t_i,\\Gamma^{ (A1g,0) })': 2,
 'B^*(x_0,t_i,t_f,\\Gamma^{ (A1g,0) })': 3,
 'B(x_1,t_i,\\Gamma^{ (Eg0,0) })': 4,
 'B^*(x_0,t_i,t_f,\\Gamma^{ (Eg0,0) })': 5,
 'B(x_1,t_i,\\Gamma^{ (Eg0,1) })': 6,
 'B^*(x_0,t_i,t_f,\\Gamma^{ (Eg0,1) })': 7,
 'B(x_1,t_i,\\Gamma^{ (Eg0,2) })': 8,
 'B^*(x_0,t_i,t_f,\\Gamma^{ (Eg0,2) })': 9,
 'B(x_1,t_i,\\Gamma^{ (Eg1,0) })': 10,
 'B^*(x_0,t_i,t_f,\\Gamma^{ (Eg1,0) })': 11,
 'B(x_1,t_i,\\Gamma^{ (Eg1,1) })': 12,
 'B^*(x_0,t_i,t_f,\\Gamma^{ (Eg1,1) })': 13,
 'B(x_1,t_i,\\Gamma^{ (Eg1,2) })': 14,
 'B^*(x_0,t_i,t_f,\\Gamma^{ (Eg1,2) })': 15}

In [36]:
allBaryonProps = {}
allBaryonSinks = {}
pIdx=0
sIdx=0
for k,v in allBaryonTensors.items():
    if k[0:3]=="B^*":
        allBaryonProps[k]=pIdx
        pIdx+=1
    else:
        allBaryonSinks[k]=sIdx
        sIdx+=1

In [37]:
allBaryonSinks

{'B(x_1,t_i,\\Gamma^{ (buchoff,0) })': 0,
 'B(x_1,t_i,\\Gamma^{ (A1g,0) })': 1,
 'B(x_1,t_i,\\Gamma^{ (Eg0,0) })': 2,
 'B(x_1,t_i,\\Gamma^{ (Eg0,1) })': 3,
 'B(x_1,t_i,\\Gamma^{ (Eg0,2) })': 4,
 'B(x_1,t_i,\\Gamma^{ (Eg1,0) })': 5,
 'B(x_1,t_i,\\Gamma^{ (Eg1,1) })': 6,
 'B(x_1,t_i,\\Gamma^{ (Eg1,2) })': 7}

In [38]:
def cppArrayPrint(lst):
    return str(lst).replace('[','{').replace(']','}')


def create_diagram_gpu_file(laphDiagrams, allBaryonTensors):
    print("file header")
    for d in laphDiagrams:
        for blocks,contraction in diagram_as_graph(d,allBaryonTensors).items():
            blocks=blocks.split(',')


            block0=allBaryonTensorsBack[int(blocks[0])]
            block1=allBaryonTensorsBack[int(blocks[1])]
            
            if block0[0:3]=="B^*":
                b0type='bprops'
                b0Idx=allBaryonProps[block0]
            else: 
                b0type='bsinks'
                b0Idx=allBaryonSinks[block0]
            
            if block1[0:3]=="B^*":
                b1type='bprops'
                b1Idx=allBaryonProps[block1]
            else:
                b1type='bsinks'
                b1Idx=allBaryonSinks[block1]

            print("diagrams.push_back(Diagram({}[{}],{}[{}],std::vector<std::vector<int>>{}));".format(b0type,b0Idx,b1type,b1Idx,cppArrayPrint(contraction)))

    print("file footer")
        

create_diagram_gpu_file(laphDiagrams,allBaryonTensors)
#for d in laphDiagrams:
    #print(diagram_as_graph(d,allBaryonTensors))
    

file header
diagrams.push_back(Diagram(bprops[0],bsinks[0],std::vector<std::vector<int>>{{3, 3}, {2, 2}, {1, 1}, {0, 0}}));
diagrams.push_back(Diagram(bprops[0],bsinks[0],std::vector<std::vector<int>>{{3, 3}, {2, 2}, {1, 0}, {0, 1}}));
diagrams.push_back(Diagram(bprops[0],bsinks[0],std::vector<std::vector<int>>{{3, 3}, {2, 0}, {1, 2}, {0, 1}}));
diagrams.push_back(Diagram(bprops[0],bsinks[0],std::vector<std::vector<int>>{{3, 3}, {2, 1}, {1, 2}, {0, 0}}));
diagrams.push_back(Diagram(bprops[0],bsinks[0],std::vector<std::vector<int>>{{3, 3}, {2, 1}, {1, 0}, {0, 2}}));
diagrams.push_back(Diagram(bprops[0],bsinks[0],std::vector<std::vector<int>>{{3, 3}, {2, 0}, {1, 1}, {0, 2}}));
diagrams.push_back(Diagram(bprops[0],bsinks[0],std::vector<std::vector<int>>{{3, 0}, {2, 3}, {1, 1}, {0, 2}}));
diagrams.push_back(Diagram(bprops[0],bsinks[0],std::vector<std::vector<int>>{{3, 1}, {2, 3}, {1, 0}, {0, 2}}));
diagrams.push_back(Diagram(bprops[0],bsinks[0],std::vector<std::vector<int>>{{3, 1}, {2, 3},

In [39]:
for d in laphDiagrams:
    pprint(d)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>